# Alzheimer’s Disease Gene Expression Analysis — GSE5281 (Polished Portfolio Notebook)

**Goal:** Build an interview-ready, reproducible workflow that downloads GEO dataset **GSE5281**, performs **quality control**, **normalization**, **exploratory analysis (PCA)**, and a basic **differential expression** example (Alzheimer’s vs Control if available), and saves results.

> This notebook is structured and commented for clarity, so that reviewers can follow your reasoning step-by-step.

## 1. Environment Setup

- Installs are here for portability (Colab/Kaggle/clean envs).
- If you prefer, move them to `requirements.txt`.

In [ ]:
# If running locally with the packages already installed, you can skip this cell.
%pip install -q GEOparse pandas numpy matplotlib seaborn scipy scikit-learn statsmodels

In [ ]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import stats
import statsmodels.stats.multitest as smm
import GEOparse

sns.set(context='notebook', style='whitegrid')
plt.rcParams['figure.figsize'] = (7, 5)
plt.rcParams['figure.dpi'] = 120

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

OUTDIR = 'outputs'
DATADIR = 'data'
os.makedirs(OUTDIR, exist_ok=True)
os.makedirs(DATADIR, exist_ok=True)

print('Environment ready.', datetime.now().isoformat())

## 2. Data Download (GEO: GSE5281)

We use `GEOparse` to download and parse **GSE5281**, a widely used Alzheimer’s disease microarray dataset. This step creates a local cache under `data/`.

In [ ]:
# Download & parse GSE5281 (cached under DATADIR)
gse = GEOparse.get_GEO('GSE5281', destdir=DATADIR, annotate_gpl=True)
print('Series:', gse.name, '| #Samples:', len(gse.gsms), '| #Platforms:', len(gse.gpls))
gpl = gse.gpls[list(gse.gpls.keys())[0]]  # select first platform

## 3. Build Expression Matrix & Sample Metadata

We pivot the sample tables into a **genes × samples** matrix using `VALUE` columns. We then try to extract **diagnosis (AD vs control)** and **brain region** from sample metadata.

In [ ]:
# Build an expression matrix: rows=probes/genes, cols=samples
expr = gse.pivot_samples('VALUE')  # (features x samples)
print('Expression matrix shape:', expr.shape)

# Clean up: drop rows with all-NA
expr = expr.dropna(how='all')
print('After dropping all-NA rows:', expr.shape)

# Basic probe annotation (if present)
annot_cols = ['ID', 'Gene Symbol', 'Gene Title', 'ENTREZ_GENE_ID', 'GENE_SYMBOL', 'GENE']
annotation = None
if hasattr(gpl.table, 'columns'):
    keep = [c for c in annot_cols if c in gpl.table.columns]
    if keep:
        annotation = gpl.table[keep].drop_duplicates(subset=[keep[0]]).set_index(keep[0])
        common = annotation.index.intersection(expr.index)
        annotation = annotation.loc[common]
        expr = expr.loc[common]
        print('Annotation merged on index; remaining shape:', expr.shape)
else:
    print('No platform table annotation found. Proceeding with probe IDs only.')

# Build sample metadata frame
def parse_characteristics(gsm):
    meta = {}
    if 'characteristics_ch1' in gsm.metadata:
        for entry in gsm.metadata['characteristics_ch1']:
            if ':' in entry:
                k, v = entry.split(':', 1)
                meta[k.strip().lower()] = v.strip()
            else:
                meta.setdefault('characteristic', []).append(entry.strip())
    if 'title' in gsm.metadata:
        meta['title'] = ' '.join(gsm.metadata['title'])
    if 'source_name_ch1' in gsm.metadata:
        meta['source_name_ch1'] = ' '.join(gsm.metadata['source_name_ch1'])
    return meta

meta_rows = []
for sid, gsm in gse.gsms.items():
    row = {'sample_id': sid}
    row.update(parse_characteristics(gsm))
    meta_rows.append(row)

meta = pd.DataFrame(meta_rows).set_index('sample_id')

def pick_first_nonnull(df, keys):
    for k in keys:
        if k in df.columns and df[k].notna().any():
            return df[k]
    return pd.Series(index=df.index, dtype='object')

candidate_dx_keys = ['diagnosis', 'disease state', 'disease', 'group', 'condition', 'phenotype']
candidate_region_keys = ['tissue', 'region', 'brain region', 'source_name_ch1']

meta['diagnosis_inferred'] = pick_first_nonnull(meta, candidate_dx_keys)
meta['region_inferred'] = pick_first_nonnull(meta, candidate_region_keys)

print('Metadata columns:', list(meta.columns))
meta.head(8)

## 4. Quality Control

- Check distributions across samples (boxplots / density).
- Inspect missingness.
- Simple log2 transform if values look unlogged.

In [ ]:
# Check if values seem logged: look at quantiles
q = expr.stack().quantile([0.01, 0.5, 0.99]).to_dict()
print('Value quantiles (1%, 50%, 99%):', q)

# Heuristic: if the 99th quantile > 100, likely not log2.
NEEDS_LOG2 = (q[0.99] > 100)
print('Apply log2(x+1)?', NEEDS_LOG2)

expr_logged = np.log2(expr + 1) if NEEDS_LOG2 else expr.copy()

# Boxplot per sample
plt.figure()
expr_logged.boxplot(rot=90)
plt.title('Per-sample distribution (log2 scale)' if NEEDS_LOG2 else 'Per-sample distribution')
plt.xlabel('Samples')
plt.ylabel('Expression')
plt.tight_layout()
plt.show()

# Missingness per sample
missing_per_sample = expr_logged.isna().mean()
plt.figure()
missing_per_sample.plot(kind='bar')
plt.title('Missing fraction per sample')
plt.ylabel('Fraction missing')
plt.tight_layout()
plt.show()

# Optional: simple imputation (median per gene)
expr_filled = expr_logged.apply(lambda row: row.fillna(row.median()), axis=1)

## 5. Exploratory Analysis — PCA

We scale features (genes) and compute PCA to visualize sample-level structure. If diagnosis labels are available, we color points by **diagnosis**; otherwise, by **brain region** (or left uncolored).

In [ ]:
# PCA on samples
scaler = StandardScaler(with_mean=True, with_std=True)
X = scaler.fit_transform(expr_filled.T)  # samples x features

pca = PCA(n_components=5, random_state=42)
PCs = pca.fit_transform(X)
pc_df = pd.DataFrame(PCs[:, :2], index=expr_filled.columns, columns=['PC1', 'PC2'])

plot_df = pc_df.join(meta[['diagnosis_inferred', 'region_inferred']], how='left')

plt.figure()
if plot_df['diagnosis_inferred'].notna().any():
    for label, sub in plot_df.groupby('diagnosis_inferred'):
        plt.scatter(sub['PC1'], sub['PC2'], label=str(label), alpha=0.8)
    plt.legend(title='Diagnosis')
elif plot_df['region_inferred'].notna().any():
    for label, sub in plot_df.groupby('region_inferred'):
        plt.scatter(sub['PC1'], sub['PC2'], label=str(label), alpha=0.8)
    plt.legend(title='Region')
else:
    plt.scatter(plot_df['PC1'], plot_df['PC2'], alpha=0.8)

plt.title('PCA of Samples (PC1 vs PC2)')
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% var)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% var)")
plt.tight_layout()
plt.show()

## 6. Differential Expression (AD vs Control, if available)

We perform a simple unpaired **t-test** for each gene between two groups.
- Groups are inferred from metadata (contains strings like `control`, `normal`, `alz`, `alzheimer`).
- P-values are corrected for multiple testing using **Benjamini–Hochberg (FDR)**.
- Results are saved to `outputs/differential_expression.csv`.

In [ ]:
# Infer two groups from diagnosis text
labels = meta['diagnosis_inferred'].astype(str).str.lower()

def label_to_group(x):
    if pd.isna(x) or x.strip() == '' or x == 'nan':
        return np.nan
    if any(k in x for k in ['control', 'normal', 'healthy', 'non-demented', 'nondemented', 'no ad']):
        return 'Control'
    if any(k in x for k in ['alzheimer', 'alz', 'ad', "alzheimer's"]):
        return 'AD'
    return np.nan

groups = labels.apply(label_to_group)
available = groups.dropna()
print('Group counts:')
print(available.value_counts())

can_do_de = (available.isin(['Control', 'AD']).sum() >= 6) and \
            (available.value_counts().get('Control', 0) >= 3) and \
            (available.value_counts().get('AD', 0) >= 3)

if not can_do_de:
    print('Not enough labeled samples for AD vs Control t-test. Skipping DE analysis.')
    de_results = pd.DataFrame()
else:
    sub_expr = expr_filled[available.index]
    sub_groups = groups.loc[available.index]

    ctrl_cols = sub_groups[sub_groups == 'Control'].index
    ad_cols = sub_groups[sub_groups == 'AD'].index

    tstats = []
    pvals = []
    log2fc = []
    for gene, row in sub_expr.iterrows():
        ctrl_vals = row[ctrl_cols].astype(float)
        ad_vals = row[ad_cols].astype(float)
        t, p = stats.ttest_ind(ad_vals, ctrl_vals, equal_var=False, nan_policy='omit')
        pvals.append(p)
        tstats.append(t)
        log2fc.append(ad_vals.mean() - ctrl_vals.mean())

    res = pd.DataFrame({
        'gene_id': sub_expr.index,
        't_stat': tstats,
        'p_value': pvals,
        'log2FC_AD_vs_Control': log2fc,
    }).set_index('gene_id')

    res['p_adj_FDR'] = smm.multipletests(res['p_value'].fillna(1.0), method='fdr_bh')[1]

    if 'annotation' in globals() and annotation is not None:
        res = res.join(annotation, how='left')

    out_path = os.path.join(OUTDIR, 'differential_expression.csv')
    res.to_csv(out_path)
    print('Saved DE results to:', out_path)

    plt.figure()
    sig = (res['p_adj_FDR'] < 0.05)
    plt.scatter(res['log2FC_AD_vs_Control'], -np.log10(res['p_value']), alpha=0.5, label='All')
    if sig.any():
        plt.scatter(res.loc[sig, 'log2FC_AD_vs_Control'], -np.log10(res.loc[sig, 'p_value']), alpha=0.8, label='FDR<0.05')
    plt.axvline(0, linestyle='--', linewidth=1)
    plt.xlabel('log2FC (AD vs Control)')
    plt.ylabel('-log10(p-value)')
    plt.title('Volcano Plot')
    plt.legend()
    plt.tight_layout()
    plt.show()

    display(res.sort_values(['p_adj_FDR', 'p_value']).head(20))
    de_results = res

## 7. Save Processed Data & Metadata

We save:
- Normalized expression matrix used for PCA/DE
- Sample metadata
- PCA coordinates

In [ ]:
expr_out = os.path.join(OUTDIR, 'expression_logged_imputed.csv')
meta_out = os.path.join(OUTDIR, 'sample_metadata_inferred.csv')
pca_out  = os.path.join(OUTDIR, 'pca_pc12.csv')

expr_filled.to_csv(expr_out)
meta.to_csv(meta_out)
if 'pc_df' in globals():
    pc_df.to_csv(pca_out)

print('Saved:')
print('-', expr_out)
print('-', meta_out)
import os
if os.path.exists(pca_out):
    print('-', pca_out)

## 8. Conclusions & Next Steps

**What we did**
- Downloaded and parsed **GSE5281** (Alzheimer’s microarray).
- Built expression matrix and inferred metadata (diagnosis/region) from characteristics.
- Ran **QC** (distributions, missingness), normalized via log2 if needed, imputed gene-wise medians.
- Performed **PCA** to explore sample structure.
- Ran a simple **differential expression** (AD vs Control) with **FDR** correction (when labels were sufficient).

**Caveats**
- `diagnosis_inferred` depends on free-text metadata and may be imperfect.
- A full analysis should include robust normalization for microarrays (e.g., RMA), batch-effect handling, covariates (age/sex/region), and model-based DE (e.g., `limma` in R).

**Next Steps**
- Stratify by **brain region** and repeat DE with linear models and covariates.
- Enrich top DE genes using GO/KEGG (e.g., `gseapy`).
- Compare findings with literature and AMP-AD resources.